In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomZoom
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

In [2]:
dataset_path = "/kaggle/input/dataset499"

In [3]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    labels="inferred",
    label_mode="binary",  # Binary classification
    batch_size=32,
    image_size=(224, 224),  # Resize images
    seed=42
)

Found 9708 files belonging to 2 classes.


In [4]:
total_batches = dataset.cardinality().numpy()
train_size = int(0.8 * total_batches)  # 80% for training
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

In [5]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1)
])

In [6]:
def build_custom_cnn(input_shape=(224, 224, 3)):
    model = models.Sequential([
        layers.Input(shape=input_shape),  # ✅ Explicit Input Layer
        
        data_augmentation,  # Augment images before CNN layers
        
        layers.Conv2D(32, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),

        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),

        layers.Flatten(),
        layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)),  # L2 Regularization
        layers.Dropout(0.6),  # Increased Dropout to prevent overfitting
        layers.Dense(1, activation='sigmoid')  # Binary classification
    ])
    
    return model


In [7]:
model = build_custom_cnn()

In [8]:
optimizer = Adam(learning_rate=0.0005)

In [9]:
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ sequential (Sequential)              │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 222, 222, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 111, 111, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 109, 109, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 54, 54, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 52, 52, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 26, 26, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 86528)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │      11,075,712 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 11,169,089 (42.61 MB)

 Trainable params: 11,169,089 (42.61 MB)

 Non-trainable params: 0 (0.00 B)

In [10]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [11]:
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=50,
    callbacks=[early_stopping]
)

Epoch 1/50
243/243 ━━━━━━━━━━━━━━━━━━━━ 22s 67ms/step - accuracy: 0.6319 - loss: 9.5051 - val_accuracy: 0.6496 - val_loss: 1.4993
Epoch 2/50
243/243 ━━━━━━━━━━━━━━━━━━━━ 16s 64ms/step - accuracy: 0.6565 - loss: 1.4092 - val_accuracy: 0.6568 - val_loss: 1.1691
Epoch 3/50
243/243 ━━━━━━━━━━━━━━━━━━━━ 15s 63ms/step - accuracy: 0.6560 - loss: 1.1439 - val_accuracy: 0.6501 - val_loss: 0.9792
Epoch 4/50
243/243 ━━━━━━━━━━━━━━━━━━━━ 15s 62ms/step - accuracy: 0.6552 - loss: 0.9766 - val_accuracy: 0.6501 - val_loss: 0.8547
Epoch 5/50
243/243 ━━━━━━━━━━━━━━━━━━━━ 15s 62ms/step - accuracy: 0.6553 - loss: 0.8671 - val_accuracy: 0.6517 - val_loss: 0.7800
Epoch 6/50
243/243 ━━━━━━━━━━━━━━━━━━━━ 15s 63ms/step - accuracy: 0.6584 - loss: 0.7945 - val_accuracy: 0.6392 - val_loss: 0.7468
Epoch 7/50
243/243 ━━━━━━━━━━━━━━━━━━━━ 15s 62ms/step - accuracy: 0.6553 - loss: 0.7526 - val_accuracy: 0.6486 - val_loss: 0.7064
Epoch 8/50
243/243 ━━━━━━━━━━━━━━━━━━━━ 15s 63ms/step - accuracy: 0.6622 - loss: 0.7165 - 